In [1]:
from __future__ import print_function
from __future__ import print_function
from __future__ import division
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import torchvision
import matplotlib.pyplot as plt
import csv
from torchvision import models
import time
import os
import copy
import cv2
import numpy as np
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
import scipy.io
import numpy as np
from pathlib import Path
import glob
import torch.utils.data as data
import pandas as pd
from PIL import Image
import os.path

PyTorch Version:  2.1.2+cu121
Torchvision Version:  0.16.2+cu121


In [2]:
IMG_EXTENSIONS = [
   '.jpg', '.JPG', '.jpeg', '.JPEG',
   '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP','.mat',
]


def is_image_file(filename):
   return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

def find_classes(dir):
   classes = os.listdir(dir)
   classes.sort()
   class_to_idx = {classes[i]: i for i in range(len(classes))}
   return classes, class_to_idx


def make_dataset(dir, class_to_idx):
   images = []
   for target in os.listdir(dir):
       d = os.path.join(dir, target)
       if not os.path.isdir(d):
           continue

       for filename in os.listdir(d):
           if is_image_file(filename):
               path = '{0}/{1}'.format(target, filename)
               #print(path)
               item = (path, class_to_idx[target])
               images.append(item)

   return images

def default_loader(path):
   return Image.open(path).convert('RGB')

def mat_loader(path):
   return scipy.io.loadmat(path)

In [3]:
classes1, class_to_idx1 = find_classes("/home/user1/icip/final_patch_ours_random/train/")
       
imgs1 = make_dataset("/home/user1/icip/final_patch_ours_random/test/", class_to_idx1)
print(class_to_idx1,len(imgs1))

print("="*100)

imgs1 = make_dataset("/home/user1/icip/final_patch_ours_similar/test/", class_to_idx1)
print(class_to_idx1,len(imgs1))

print("="*100)

imgs1 = make_dataset("/home/user1/icip/final_patch_merged_ours/test/", class_to_idx1)
print(class_to_idx1,len(imgs1))

{'D01_Samsung_Galaxy_S20Plus': 0, 'D02_Nothing_One': 1, 'D03_Samsung_Galaxy_A03': 2, 'D04_Samsung_Galaxy_M04': 3, 'D05_Vivo_V9_Pro': 4, 'D06_Apple_Iphone_12Mini': 5, 'D07_Apple_Iphone_11': 6, 'D08_Redmi_Note_8Pro': 7, 'D09_Samsung_Galaxy_J8_10G': 8, 'D10_Samsung_Galaxy_F41': 9, 'D11_OnePlus_8T': 10, 'D12_Vivo_Y02t': 11, 'D13_Oppo_A17k': 12, 'D14_Samsung_Galaxy_S20FE': 13, 'D15_Motorola_Motog60': 14, 'D16_Samsung_Galaxy_S21FE': 15, 'D17_Apple_Iphone_12': 16, 'D18_IQOO_Z3': 17, 'D19_IQOO_Z6_Lite': 18, 'D20_Motorola_MotoG73_5G': 19, 'D21_OnePlus_10Pro_5G': 20, 'D22_Poco_F5': 21, 'D23_Poco_F5_Pro_5G': 22, 'D24_Realme_8': 23, 'D25_Realme_X3_Superzoom': 24, 'D26_Redmi_9i_Sport': 25, 'D27_Redmi_Note10_Pro': 26, 'D28_Apple_Iphone_13': 27, 'D29_Apple_Iphone_15': 28, 'D30_Vivo_Y75': 29} 274176
{'D01_Samsung_Galaxy_S20Plus': 0, 'D02_Nothing_One': 1, 'D03_Samsung_Galaxy_A03': 2, 'D04_Samsung_Galaxy_M04': 3, 'D05_Vivo_V9_Pro': 4, 'D06_Apple_Iphone_12Mini': 5, 'D07_Apple_Iphone_11': 6, 'D08_Redmi_No

In [4]:
class ImageFolderLoader(data.Dataset):
   def __init__(self, root1,transform_1=None,
                target_transform=None,
                loader=default_loader):
       classes1, class_to_idx1 = find_classes(root1)
       
       imgs1 = make_dataset(root1, class_to_idx1)
      

       self.root1 = root1
       self.imgs1 = imgs1
       self.classes1 = classes1
       self.class_to_idx1 = class_to_idx1
       self.transform_1 = transform_1
       self.target_transform = target_transform
       self.loader = loader
        
       self.img_noise = None
       self.img_rgb = None
    
    
   def SRM(self):
    
        imgs = self.img_rgb
        
        filter2 = [[0, 0, 0, 0, 0],
                   [0, -1, 2, -1, 0],
                   [0, 2, -4, 2, 0],
                   [0, -1, 2, -1, 0],
                   [0, 0, 0, 0, 0]]
        # filter2：egde5*5
        filter1 = [[-1, 2, -2, 2, -1],
                   [2, -6, 8, -6, 2],
                   [-2, 8, -12, 8, -2],
                   [2, -6, 8, -6, 2],
                   [-1, 2, -2, 2, -1]]
        # filter3
        filter3 = [[0, 0, 0, 0, 0],
                   [0, 0, 1, 0, 0],
                   [0, 0,-2, 0, 0],
                   [0, 0, 1, 0, 0],
                   [0, 0, 0, 0, 0]]

        filter1 = np.asarray(filter1, dtype=float) / 12
        filter2 = np.asarray(filter2, dtype=float) / 4
        filter3 = np.asarray(filter3, dtype=float) / 2

        filters = [[filter1, filter1, filter1], [filter2, filter2, filter2], [filter3, filter3, filter3]]# (3,3,5,5)

        filters = torch.FloatTensor(filters)  
         # (3,3,5,5)
        imgs = np.array(imgs, dtype=float)  # (375,500,3)
        w,h,c = imgs.shape
        imgs = imgs.reshape(1,w,h,c)
        imgs = np.einsum('klij->kjli', imgs)

        
        input = torch.tensor(imgs, dtype=torch.float32)


        op1 = F.conv2d(input, filters, stride=1, padding=2)
        #print('op1\'s shape', op1.shape)

        
        op1= op1.reshape(c,w,h)
    
        self.img_noise = op1
        
        
        
       
       

   def __getitem__(self, index):
    

       path1, target1 = self.imgs1[index]  
       filename = Path(path1).stem 
       img1 = self.loader(os.path.join(self.root1, path1))  
    
       self.img_rgb = img1
       
       
       self.SRM()
    
       #print(self.img_noise.shape)
       
    

       if self.transform_1 is not None:
           img1 = self.transform_1(self.img_rgb)
        
       if self.target_transform is not None:
           target1 = self.target_transform(target)
            
       
       
       return img1,self.img_noise, target1,filename

   def __len__(self):
       return len(self.imgs1)

In [5]:
data_transforms = transforms.Compose([
transforms.ToTensor()
])

batchsize=1

val_dataset_random = ImageFolderLoader(
        "/home/user1/icip/final_patch_ours_random/test/",
        transform_1=data_transforms
    )

test_loader_random = torch.utils.data.DataLoader(
        val_dataset_random, batch_size=batchsize,
        shuffle=False, num_workers=4
  )

val_dataset_similar = ImageFolderLoader(
        "/home/user1/icip/final_patch_ours_similar/test/",
        transform_1=data_transforms
    )

test_loader_similar = torch.utils.data.DataLoader(
        val_dataset_similar, batch_size=batchsize,
        shuffle=False, num_workers=4
  )

val_dataset_merged = ImageFolderLoader(
        "/home/user1/icip/final_patch_merged_ours/test/",
        transform_1=data_transforms
    )

test_loader_merged = torch.utils.data.DataLoader(
        val_dataset_merged, batch_size=batchsize,
        shuffle=False, num_workers=4
  )

In [6]:
len(val_dataset_random), len(val_dataset_similar), len(val_dataset_merged)

(274176, 245760, 519936)

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        resnet1 = models.resnet50(pretrained=True)
        modules1 = list(resnet1.children())[:-1]      # delete the last fc layer.
        self.resnet1 = nn.Sequential(*modules1)

        resnet2 = models.resnet50(pretrained=True)
        modules2 = list(resnet2.children())[:-1] 
        self.resnet2 = nn.Sequential(*modules2)
        
        
    
        self.fc1 = nn.Linear(2048*2, 2048)
        self.fc2 = nn.Linear(2048,30)
        
        
        self.logsoftmax = nn.LogSoftmax(dim=1)
        
       

    def forward(self, x1,x2):
        
        x1 = self.resnet1(x1)
        x1 = x1.view(x1.size(0), -1)

        x2 = self.resnet2(x2)
        x2 = x2.view(x2.size(0), -1)
        

        x = torch.cat((x1,x2),dim=1)

        x = self.fc1(x)
        x = self.fc2(x)
       
        out_fc = x
        output = self.logsoftmax(x)
        
        return output, out_fc

In [8]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0')
model = Net().to(device)

/home/user1/miniconda3/envs/torch/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/user1/miniconda3/envs/torch/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
model = torch.load("Rana_trained_model_random_final")
model.eval()

results_folder = "Results_Random"
if os.path.exists(results_folder):
    # If it exists, delete the folder and its content
    print("Deleting existing Results_Random folder...")
    for file in os.listdir(results_folder):
        file_path = os.path.join(results_folder, file)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                os.rmdir(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

os.makedirs(results_folder, exist_ok=True)

a = -1

with torch.no_grad():
    for batch_idx, (imgs1,imgs2,labels1,patch_filename) in enumerate(test_loader_random):
        c = labels1
        d = c.cpu().numpy()[0]

        if(d!=a):
            print("Yes_Class",d)
            a= d
            z = d
            file_class = os.path.join(results_folder, f"Test_Class_{z}.csv")

            with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Batch_Id","Patch_Filename","True Class","Predicted Class","Probability of Predicted Class"])
    


        img_org, mat_img, target = imgs1.to(device),imgs2.to(device), labels1.to(device)
        output, fc_feature = model(img_org,mat_img)
        
        output = F.softmax(fc_feature,dim=1)
        

        actual = target
        _, predicted = torch.max(output.data, 1)

        y_true = actual.cpu().numpy()[0]
        y_pred =predicted.cpu().numpy()[0]

        prob_y_pred = output[0][y_pred]
        prob_y_pred = prob_y_pred.cpu().numpy()
        prob_y_pred = np.around(prob_y_pred,decimals=2)

        if(batch_idx % 1000 == 0):
          print(batch_idx,patch_filename,y_true,y_pred,prob_y_pred)
        
        
        with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([batch_idx,patch_filename,y_true,y_pred,prob_y_pred])

Deleting existing Results_Random folder...


/tmp/ipykernel_26637/248763931.py:50: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  filters = torch.FloatTensor(filters)
/tmp/ipykernel_26637/248763931.py:50: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  filters = torch.FloatTensor(filters)
/tmp/ipykernel_26637/248763931.py:50: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  filters = torch.FloatTensor(filt

Yes_Class 0
0 ('D01_rnd_34_15',) 0 0 1.0
1000 ('D01_rnd_63_230',) 0 0 1.0
2000 ('D01_rnd_53_9',) 0 0 1.0
3000 ('D01_rnd_34_253',) 0 0 0.99
4000 ('D01_rnd_99_107',) 0 0 0.99
5000 ('D01_rnd_150_210',) 0 0 1.0
6000 ('D01_rnd_99_80',) 0 0 1.0
7000 ('D01_rnd_150_69',) 0 0 1.0
8000 ('D01_rnd_153_244',) 0 0 1.0
Yes_Class 28
9000 ('D29_rnd_146_246',) 28 28 1.0
10000 ('D29_rnd_98_231',) 28 14 0.51
11000 ('D29_rnd_2_42',) 28 28 1.0
12000 ('D29_rnd_39_122',) 28 28 0.52
13000 ('D29_rnd_113_229',) 28 28 1.0
14000 ('D29_rnd_150_12',) 28 28 1.0
15000 ('D29_rnd_141_4',) 28 28 1.0
16000 ('D29_rnd_119_252',) 28 28 1.0
17000 ('D29_rnd_119_231',) 28 28 1.0
Yes_Class 22
18000 ('D23_rnd_87_69',) 22 22 1.0
19000 ('D23_rnd_135_79',) 22 22 1.0
20000 ('D23_rnd_124_54',) 22 22 0.98
21000 ('D23_rnd_70_54',) 22 22 1.0
22000 ('D23_rnd_61_55',) 22 22 1.0
23000 ('D23_rnd_135_55',) 22 22 1.0
24000 ('D23_rnd_99_184',) 22 22 1.0
25000 ('D23_rnd_113_241',) 22 22 1.0
Yes_Class 27
26000 ('D28_rnd_60_49',) 27 28 1.0
27000 (

221000 ('D07_rnd_176_216',) 6 6 1.0
222000 ('D07_rnd_24_32',) 6 6 1.0
223000 ('D07_rnd_108_207',) 6 6 1.0
Yes_Class 16
224000 ('D17_rnd_98_206',) 16 16 1.0
225000 ('D17_rnd_64_131',) 16 16 1.0
226000 ('D17_rnd_124_196',) 16 16 1.0
227000 ('D17_rnd_110_173',) 16 16 1.0
228000 ('D17_rnd_135_98',) 16 16 1.0
229000 ('D17_rnd_113_100',) 16 16 1.0
230000 ('D17_rnd_110_137',) 16 16 1.0
231000 ('D17_rnd_10_121',) 16 16 1.0
Yes_Class 26
232000 ('D27_rnd_47_94',) 26 26 1.0
233000 ('D27_rnd_2_172',) 26 26 1.0
234000 ('D27_rnd_99_205',) 26 26 1.0
235000 ('D27_rnd_99_86',) 26 26 1.0
236000 ('D27_rnd_75_173',) 26 26 1.0
237000 ('D27_rnd_153_254',) 26 26 1.0
238000 ('D27_rnd_99_201',) 26 26 1.0
239000 ('D27_rnd_47_116',) 26 26 1.0
Yes_Class 11
240000 ('D12_rnd_117_180',) 11 11 1.0
241000 ('D12_rnd_129_204',) 11 11 1.0
242000 ('D12_rnd_71_121',) 11 11 1.0
243000 ('D12_rnd_176_87',) 11 11 1.0
244000 ('D12_rnd_150_124',) 11 11 1.0
245000 ('D12_rnd_176_102',) 11 11 1.0
246000 ('D12_rnd_70_195',) 11 11 1.

In [10]:
csv_dir = img_dir = "Results_Random/"
data_path = os.path.join(img_dir,'*csv')
files = glob.glob(data_path)

with open(os.path.join(csv_dir, 'Image_Level_Results.csv'), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Image Class Label","Number of Images",\
                         "Correct Predicted Images",\
                         "Number of Patches",\
                         "Total Patches(Correct Classified Images)",\
                         "Correct Predicted Patches(Correct Classified Images)",\
                         "Precetange Votes Per Image(Only Correct Images)",\
                         "Average Softmax Probability of Correct Patch(Only Correct Images)"])

In [11]:
for f in files:
      df = pd.read_csv(f)
      data = df.sort_values(by=['Patch_Filename'])
      classname = Path(f).stem
      classname = int(classname.split("_")[2])
      a="a"

      true_image_class = classname
      total_images_perclass = 0
      correct_images_perclass = 0
      total_patches_perclass = 0

      total_class_votes =0
      total_class_patches =0
      prob_avg_correct_patch = 0.0
      votes = 0

      arr_pred_patches = []
      arr_pred_patches_prob = []
      total_correc_img_patches = 0


      for ind in data.index:
              filename = df['Patch_Filename'][ind]
              pred_patch_class = df['Predicted Class'][ind]
              pred_patch_prob = df['Probability of Predicted Class'][ind]
              
              filename = filename.split("_")
              file_length = len(filename)
              initial_filename = filename[:-1]
              patch_name = filename[file_length-1]
              #print(initial_filename)
              #print(patch_name)

              if(a!= initial_filename and a=="a"):
                  a = initial_filename



              if(a!=initial_filename and a!="a"):
                  total_images_perclass = total_images_perclass + 1

                  counts = np.bincount(arr_pred_patches)
                  pred_image_class = np.argmax(counts)
                  votes = np.count_nonzero(arr_pred_patches==pred_image_class)
                  s=0
                  
                  if(pred_image_class == true_image_class):
                      correct_images_perclass = correct_images_perclass + 1
                      total_class_votes = total_class_votes + votes
                      total_class_patches = total_class_patches + len(arr_pred_patches)
                      
                      total_correc_img_patches = total_correc_img_patches + len(arr_pred_patches)

                      z = np.where(np.array(arr_pred_patches)==true_image_class,1,0)
                      for i in range(0,len(arr_pred_patches)):
                          if(z[i]==1):
                              s= s+1
                              prob_avg_correct_patch = prob_avg_correct_patch + arr_pred_patches_prob[i]
              
                  arr_pred_patches = []
                  arr_pred_patches_prob = []
                  a = initial_filename



              if(a==initial_filename):
                  total_patches_perclass = total_patches_perclass + 1
                  arr_pred_patches.append(pred_patch_class)
                  arr_pred_patches_prob.append(pred_patch_prob)



      total_images_perclass = total_images_perclass + 1

      counts = np.bincount(arr_pred_patches)
      pred_image_class = np.argmax(counts)
      votes = np.count_nonzero(arr_pred_patches==pred_image_class)

      if(pred_image_class == true_image_class):
          correct_images_perclass = correct_images_perclass + 1
          total_class_votes = total_class_votes + votes
          total_class_patches = total_class_patches + len(arr_pred_patches)
          
          total_correc_img_patches = total_correc_img_patches + len(arr_pred_patches)

          z = np.where(np.array(arr_pred_patches)==true_image_class,1,0)
          for i in range(0,len(arr_pred_patches)):
              if(z[i]==1):
                  prob_avg_correct_patch = prob_avg_correct_patch + arr_pred_patches_prob[i]

      arr_pred_patches = []
      arr_pred_patches_prob = []
      
      if(correct_images_perclass!=0):
          prob_avg_correct_patch = prob_avg_correct_patch / total_class_votes
          prob_avg_correct_patch = np.around(prob_avg_correct_patch,decimals=2)
          avg_vote_perclass = np.around((total_class_votes*100) / total_correc_img_patches,decimals=2)
          print(total_images_perclass,total_patches_perclass,correct_images_perclass,total_class_votes,avg_vote_perclass,prob_avg_correct_patch)  

          with open(os.path.join(csv_dir + 'Image_Level_Results.csv'), 'a+', newline='') as file:
              writer = csv.writer(file)
              writer.writerow([classname,total_images_perclass,correct_images_perclass,total_patches_perclass,\
                              total_correc_img_patches,total_class_votes,\
                              avg_vote_perclass,\
                              prob_avg_correct_patch])
      else:
          with open(os.path.join(csv_dir + 'Image_Level_Results.csv'), 'a+', newline='') as file:
              writer = csv.writer(file)
              writer.writerow([classname,total_images_perclass,correct_images_perclass,total_patches_perclass,\
                              total_correc_img_patches,total_class_votes,\
                              avg_vote_perclass,\
                              prob_avg_correct_patch])
              
              
df = pd.read_csv(os.path.join(csv_dir + 'Image_Level_Results.csv'))

ILA_random = sum(df['Correct Predicted Images']) / sum(df['Number of Images']) * 100

print(f"Image Level Accuracy: {ILA_random}%")

36 9216 36 8543 92.7 0.99
32 8192 32 8005 97.72 1.0
43 11008 43 10934 99.33 1.0
38 9728 37 9266 97.83 0.99
32 8192 32 7813 95.37 0.99
34 8704 34 8523 97.92 1.0
32 8192 32 7957 97.13 1.0
40 10240 39 9817 98.33 1.0
44 11264 44 10742 95.37 0.99
47 12032 47 11791 98.0 1.0
35 8960 33 8143 96.39 0.99
31 7936 31 7685 96.84 1.0
34 8704 34 8234 94.6 0.99
34 8704 34 8464 97.24 0.99
33 8448 33 8388 99.29 1.0
31 7936 31 7329 92.35 0.99
34 8704 34 8463 97.23 0.99
31 7936 31 7155 90.16 0.99
31 7936 31 7831 98.68 1.0
56 14336 56 13813 96.35 0.99
51 13056 51 13030 99.8 1.0
33 8448 33 7991 94.59 0.99
31 7936 30 7038 91.64 0.99
33 8448 33 8073 95.56 0.99
31 7936 27 5918 85.62 0.98
30 7680 30 6989 91.0 0.98
32 8192 31 7616 95.97 0.99
32 8192 32 7943 96.96 0.99
35 8960 34 8304 95.4 0.99
35 8960 34 8425 96.79 0.99
Image Level Accuracy: 98.8795518207283%


In [12]:
model.eval()

results_folder = "Results_Similar"
if os.path.exists(results_folder):
    # If it exists, delete the folder and its content
    print("Deleting existing Results_Similar folder...")
    for file in os.listdir(results_folder):
        file_path = os.path.join(results_folder, file)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                os.rmdir(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

os.makedirs(results_folder, exist_ok=True)

a = -1

with torch.no_grad():
    for batch_idx, (imgs1,imgs2,labels1,patch_filename) in enumerate(test_loader_similar):
        c = labels1
        d = c.cpu().numpy()[0]

        if(d!=a):
            print("Yes_Class",d)
            a= d
            z = d
            file_class = os.path.join(results_folder, f"Test_Class_{z}.csv")

            with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Batch_Id","Patch_Filename","True Class","Predicted Class","Probability of Predicted Class"])
    


        img_org, mat_img, target = imgs1.to(device),imgs2.to(device), labels1.to(device)
        output, fc_feature = model(img_org,mat_img)
        
        output = F.softmax(fc_feature,dim=1)
        

        actual = target
        _, predicted = torch.max(output.data, 1)

        y_true = actual.cpu().numpy()[0]
        y_pred =predicted.cpu().numpy()[0]

        prob_y_pred = output[0][y_pred]
        prob_y_pred = prob_y_pred.cpu().numpy()
        prob_y_pred = np.around(prob_y_pred,decimals=2)

        if(batch_idx % 1000 == 0):
          print(batch_idx,patch_filename,y_true,y_pred,prob_y_pred)
        
        
        with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([batch_idx,patch_filename,y_true,y_pred,prob_y_pred])

Deleting existing Results_Similar folder...


/tmp/ipykernel_26637/248763931.py:50: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  filters = torch.FloatTensor(filters)
/tmp/ipykernel_26637/248763931.py:50: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  filters = torch.FloatTensor(filters)
/tmp/ipykernel_26637/248763931.py:50: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  filters = torch.FloatTensor(filt

Yes_Class 0
0 ('D01_color_2_167',) 0 0 1.0
1000 ('D01_obj_29_58',) 0 3 0.94
2000 ('D01_tex_25_97',) 0 15 1.0
3000 ('D01_nat_2_131',) 0 0 1.0
4000 ('D01_nat_39_61',) 0 0 1.0
5000 ('D01_nat_1_8',) 0 0 1.0
6000 ('D01_tex_36_229',) 0 26 0.99
7000 ('D01_obj_36_26',) 0 28 1.0
8000 ('D01_tex_25_239',) 0 15 1.0
Yes_Class 28
9000 ('D29_obj_17_207',) 28 6 0.72
10000 ('D29_nat_9_82',) 28 10 0.84
11000 ('D29_tex_39_254',) 28 15 0.81
12000 ('D29_nat_9_109',) 28 28 0.98
13000 ('D29_nat_49_206',) 28 10 1.0
14000 ('D29_obj_17_86',) 28 5 0.99
15000 ('D29_obj_36_109',) 28 28 1.0
16000 ('D29_obj_30_181',) 28 27 0.96
Yes_Class 22
17000 ('D23_color_14_164',) 22 22 1.0
18000 ('D23_tex_25_176',) 22 22 1.0
19000 ('D23_obj_36_156',) 22 22 0.7
20000 ('D23_nat_17_19',) 22 22 1.0
21000 ('D23_obj_36_203',) 22 22 1.0
22000 ('D23_obj_2_17',) 22 22 1.0
23000 ('D23_color_14_94',) 22 26 0.68
24000 ('D23_nat_30_231',) 22 22 0.98
Yes_Class 27
25000 ('D28_nat_9_129',) 27 5 0.5
26000 ('D28_nat_29_191',) 27 27 0.98
27000 ('

221000 ('D12_nat_1_36',) 11 11 0.89
Yes_Class 15
222000 ('D16_nat_49_125',) 15 15 1.0
223000 ('D16_nat_45_215',) 15 15 1.0
224000 ('D16_obj_5_100',) 15 15 0.56
225000 ('D16_obj_25_194',) 15 15 0.71
226000 ('D16_tex_36_63',) 15 15 1.0
227000 ('D16_nat_1_185',) 15 15 1.0
228000 ('D16_obj_25_109',) 15 15 0.99
229000 ('D16_tex_36_79',) 15 15 1.0
Yes_Class 24
230000 ('D25_nat_39_133',) 24 24 1.0
231000 ('D25_tex_2_186',) 24 26 0.99
232000 ('D25_color_14_226',) 24 17 1.0
233000 ('D25_nat_39_37',) 24 1 0.69
234000 ('D25_nat_9_14',) 24 24 1.0
235000 ('D25_nat_29_14',) 24 24 1.0
236000 ('D25_tex_1_38',) 24 24 1.0
237000 ('D25_nat_30_7',) 24 24 1.0
Yes_Class 8
238000 ('D09_nat_25_181',) 8 8 1.0
239000 ('D09_nat_49_139',) 8 8 1.0
240000 ('D09_nat_36_131',) 8 8 1.0
241000 ('D09_tex_7_35',) 8 8 1.0
242000 ('D09_obj_1_8',) 8 8 1.0
243000 ('D09_nat_45_21',) 8 8 1.0
244000 ('D09_nat_25_174',) 8 8 0.97
245000 ('D09_color_14_122',) 8 8 1.0


In [13]:
csv_dir = img_dir = "Results_Similar/"
data_path = os.path.join(img_dir,'*csv')
files = glob.glob(data_path)

with open(os.path.join(csv_dir, 'Image_Level_Results.csv'), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Image Class Label","Number of Images",\
                         "Correct Predicted Images",\
                         "Number of Patches",\
                         "Total Patches(Correct Classified Images)",\
                         "Correct Predicted Patches(Correct Classified Images)",\
                         "Precetange Votes Per Image(Only Correct Images)",\
                         "Average Softmax Probability of Correct Patch(Only Correct Images)"])

In [14]:
for f in files:
      df = pd.read_csv(f)
      data = df.sort_values(by=['Patch_Filename'])
      classname = Path(f).stem
      classname = int(classname.split("_")[2])
      a="a"

      true_image_class = classname
      total_images_perclass = 0
      correct_images_perclass = 0
      total_patches_perclass = 0

      total_class_votes =0
      total_class_patches =0
      prob_avg_correct_patch = 0.0
      votes = 0

      arr_pred_patches = []
      arr_pred_patches_prob = []
      total_correc_img_patches = 0


      for ind in data.index:
              filename = df['Patch_Filename'][ind]
              pred_patch_class = df['Predicted Class'][ind]
              pred_patch_prob = df['Probability of Predicted Class'][ind]
              
              filename = filename.split("_")
              file_length = len(filename)
              initial_filename = filename[:-1]
              patch_name = filename[file_length-1]
              #print(initial_filename)
              #print(patch_name)

              if(a!= initial_filename and a=="a"):
                  a = initial_filename



              if(a!=initial_filename and a!="a"):
                  total_images_perclass = total_images_perclass + 1

                  counts = np.bincount(arr_pred_patches)
                  pred_image_class = np.argmax(counts)
                  votes = np.count_nonzero(arr_pred_patches==pred_image_class)
                  s=0
                  
                  if(pred_image_class == true_image_class):
                      correct_images_perclass = correct_images_perclass + 1
                      total_class_votes = total_class_votes + votes
                      total_class_patches = total_class_patches + len(arr_pred_patches)
                      
                      total_correc_img_patches = total_correc_img_patches + len(arr_pred_patches)

                      z = np.where(np.array(arr_pred_patches)==true_image_class,1,0)
                      for i in range(0,len(arr_pred_patches)):
                          if(z[i]==1):
                              s= s+1
                              prob_avg_correct_patch = prob_avg_correct_patch + arr_pred_patches_prob[i]
              
                  arr_pred_patches = []
                  arr_pred_patches_prob = []
                  a = initial_filename



              if(a==initial_filename):
                  total_patches_perclass = total_patches_perclass + 1
                  arr_pred_patches.append(pred_patch_class)
                  arr_pred_patches_prob.append(pred_patch_prob)



      total_images_perclass = total_images_perclass + 1

      counts = np.bincount(arr_pred_patches)
      pred_image_class = np.argmax(counts)
      votes = np.count_nonzero(arr_pred_patches==pred_image_class)

      if(pred_image_class == true_image_class):
          correct_images_perclass = correct_images_perclass + 1
          total_class_votes = total_class_votes + votes
          total_class_patches = total_class_patches + len(arr_pred_patches)
          
          total_correc_img_patches = total_correc_img_patches + len(arr_pred_patches)

          z = np.where(np.array(arr_pred_patches)==true_image_class,1,0)
          for i in range(0,len(arr_pred_patches)):
              if(z[i]==1):
                  prob_avg_correct_patch = prob_avg_correct_patch + arr_pred_patches_prob[i]

      arr_pred_patches = []
      arr_pred_patches_prob = []
      
      if(correct_images_perclass!=0):
          prob_avg_correct_patch = prob_avg_correct_patch / total_class_votes
          prob_avg_correct_patch = np.around(prob_avg_correct_patch,decimals=2)
          avg_vote_perclass = np.around((total_class_votes*100) / total_correc_img_patches,decimals=2)
          print(total_images_perclass,total_patches_perclass,correct_images_perclass,total_class_votes,avg_vote_perclass,prob_avg_correct_patch)  

          with open(os.path.join(csv_dir + 'Image_Level_Results.csv'), 'a+', newline='') as file:
              writer = csv.writer(file)
              writer.writerow([classname,total_images_perclass,correct_images_perclass,total_patches_perclass,\
                              total_correc_img_patches,total_class_votes,\
                              avg_vote_perclass,\
                              prob_avg_correct_patch])
      else:
          with open(os.path.join(csv_dir + 'Image_Level_Results.csv'), 'a+', newline='') as file:
              writer = csv.writer(file)
              writer.writerow([classname,total_images_perclass,correct_images_perclass,total_patches_perclass,\
                              total_correc_img_patches,total_class_votes,\
                              avg_vote_perclass,\
                              prob_avg_correct_patch])
              
              
df = pd.read_csv(os.path.join(csv_dir + 'Image_Level_Results.csv'))

ILA_similar = sum(df['Correct Predicted Images']) / sum(df['Number of Images']) * 100

print(f"Image Level Accuracy: {ILA_similar}%")

32 8192 26 5674 85.25 0.97
32 8192 28 6038 84.24 0.97
32 8192 28 6316 88.11 0.98
32 8192 9 1951 84.68 0.99
32 8192 31 7381 93.01 0.99
32 8192 24 5245 85.37 0.98
32 8192 32 8134 99.29 1.0
32 8192 31 6911 87.08 0.98
32 8192 26 5940 89.24 0.98
32 8192 31 6848 86.29 0.98
32 8192 15 3205 83.46 0.98
32 8192 31 6882 86.72 0.98
32 8192 29 5857 78.89 0.97
32 8192 31 7211 90.86 0.98
32 8192 30 7079 92.17 0.98
32 8192 22 4079 72.43 0.96
32 8192 22 4908 87.14 0.97
32 8192 23 3614 61.38 0.95
32 8192 24 5732 93.29 0.99
32 8192 28 6618 92.33 0.99
32 8192 29 7192 96.88 1.0
32 8192 26 5740 86.24 0.98
32 8192 17 3312 76.1 0.95
32 8192 30 6665 86.78 0.98
32 8192 21 3902 72.58 0.96
32 8192 25 3648 57.0 0.93
32 8192 29 6887 92.77 0.99
32 8192 20 3402 66.45 0.94
32 8192 31 6788 85.53 0.97
32 8192 21 4833 89.9 0.98
Image Level Accuracy: 80.20833333333334%


In [15]:
model.eval()

results_folder = "Results_Merged"
if os.path.exists(results_folder):
    # If it exists, delete the folder and its content
    print("Deleting existing Results_Merged folder...")
    for file in os.listdir(results_folder):
        file_path = os.path.join(results_folder, file)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                os.rmdir(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

os.makedirs(results_folder, exist_ok=True)

a = -1

with torch.no_grad():
    for batch_idx, (imgs1,imgs2,labels1,patch_filename) in enumerate(test_loader_merged):
        c = labels1
        d = c.cpu().numpy()[0]

        if(d!=a):
            print("Yes_Class",d)
            a= d
            z = d
            file_class = os.path.join(results_folder, f"Test_Class_{z}.csv")

            with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Batch_Id","Patch_Filename","True Class","Predicted Class","Probability of Predicted Class"])
    


        img_org, mat_img, target = imgs1.to(device),imgs2.to(device), labels1.to(device)
        output, fc_feature = model(img_org,mat_img)
        
        output = F.softmax(fc_feature,dim=1)
        

        actual = target
        _, predicted = torch.max(output.data, 1)

        y_true = actual.cpu().numpy()[0]
        y_pred =predicted.cpu().numpy()[0]

        prob_y_pred = output[0][y_pred]
        prob_y_pred = prob_y_pred.cpu().numpy()
        prob_y_pred = np.around(prob_y_pred,decimals=2)

        if(batch_idx % 1000 == 0):
          print(batch_idx,patch_filename,y_true,y_pred,prob_y_pred)
        
        
        with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([batch_idx,patch_filename,y_true,y_pred,prob_y_pred])

Deleting existing Results_Merged folder...


/tmp/ipykernel_26637/248763931.py:50: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  filters = torch.FloatTensor(filters)
/tmp/ipykernel_26637/248763931.py:50: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  filters = torch.FloatTensor(filters)
/tmp/ipykernel_26637/248763931.py:50: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  filters = torch.FloatTensor(filt

Yes_Class 0
0 ('D01_rnd_34_15',) 0 0 1.0
1000 ('D01_nat_39_179',) 0 0 0.94
2000 ('D01_rnd_160_33',) 0 0 1.0
3000 ('D01_color_2_66',) 0 15 1.0
4000 ('D01_rnd_141_63',) 0 0 1.0
5000 ('D01_tex_7_230',) 0 13 1.0
6000 ('D01_rnd_99_27',) 0 0 1.0
7000 ('D01_tex_29_111',) 0 0 1.0
8000 ('D01_nat_30_173',) 0 13 1.0
9000 ('D01_tex_7_224',) 0 0 1.0
10000 ('D01_obj_29_123',) 0 29 0.72
11000 ('D01_obj_29_155',) 0 3 0.5
12000 ('D01_tex_17_89',) 0 0 0.88
13000 ('D01_rnd_84_69',) 0 0 1.0
14000 ('D01_nat_1_47',) 0 0 1.0
15000 ('D01_rnd_86_150',) 0 0 1.0
16000 ('D01_obj_39_197',) 0 0 0.99
Yes_Class 28
17000 ('D29_obj_25_106',) 28 3 0.77
18000 ('D29_rnd_141_230',) 28 28 1.0
19000 ('D29_tex_25_227',) 28 28 1.0
20000 ('D29_rnd_98_217',) 28 28 1.0
21000 ('D29_rnd_7_240',) 28 28 1.0
22000 ('D29_rnd_122_207',) 28 28 1.0
23000 ('D29_rnd_83_76',) 28 28 1.0
24000 ('D29_nat_9_192',) 28 28 1.0
25000 ('D29_obj_2_179',) 28 28 0.94
26000 ('D29_rnd_98_202',) 28 28 1.0
27000 ('D29_nat_1_160',) 28 28 1.0
28000 ('D29_rnd_

224000 ('D20_rnd_82_180',) 19 19 1.0
225000 ('D20_tex_17_16',) 19 19 0.99
226000 ('D20_tex_39_10',) 19 19 1.0
227000 ('D20_rnd_110_149',) 19 19 0.99
228000 ('D20_rnd_70_132',) 19 19 1.0
229000 ('D20_rnd_130_191',) 19 19 1.0
Yes_Class 5
230000 ('D06_rnd_50_56',) 5 5 1.0
231000 ('D06_nat_29_180',) 5 26 1.0
232000 ('D06_rnd_50_101',) 5 5 1.0
233000 ('D06_color_14_106',) 5 5 1.0
234000 ('D06_nat_30_102',) 5 27 0.82
235000 ('D06_rnd_132_81',) 5 5 1.0
236000 ('D06_rnd_129_67',) 5 5 1.0
237000 ('D06_rnd_16_193',) 5 5 1.0
238000 ('D06_nat_36_45',) 5 6 0.99
239000 ('D06_obj_30_181',) 5 27 1.0
240000 ('D06_rnd_99_102',) 5 5 1.0
241000 ('D06_tex_7_85',) 5 5 0.96
242000 ('D06_rnd_64_143',) 5 5 1.0
243000 ('D06_rnd_138_162',) 5 5 0.97
244000 ('D06_rnd_53_101',) 5 5 1.0
245000 ('D06_tex_2_255',) 5 6 0.49
Yes_Class 1
246000 ('D02_rnd_215_196',) 1 1 1.0
247000 ('D02_nat_49_108',) 1 1 1.0
248000 ('D02_nat_29_164',) 1 1 1.0
249000 ('D02_rnd_215_20',) 1 1 1.0
250000 ('D02_rnd_99_135',) 1 1 1.0
251000 ('D

447000 ('D27_rnd_108_239',) 26 26 1.0
448000 ('D27_obj_1_140',) 26 26 1.0
449000 ('D27_rnd_36_34',) 26 26 1.0
450000 ('D27_rnd_25_224',) 26 26 0.99
451000 ('D27_rnd_75_19',) 26 20 0.8
452000 ('D27_tex_17_145',) 26 26 1.0
Yes_Class 11
453000 ('D12_tex_30_62',) 11 15 0.99
454000 ('D12_rnd_46_6',) 11 11 1.0
455000 ('D12_tex_29_80',) 11 29 0.97
456000 ('D12_obj_2_231',) 11 28 0.92
457000 ('D12_nat_45_232',) 11 11 1.0
458000 ('D12_rnd_36_106',) 11 11 1.0
459000 ('D12_rnd_36_185',) 11 11 1.0
460000 ('D12_rnd_188_5',) 11 11 1.0
461000 ('D12_obj_5_62',) 11 12 0.99
462000 ('D12_rnd_99_17',) 11 11 1.0
463000 ('D12_rnd_57_59',) 11 11 1.0
464000 ('D12_rnd_160_123',) 11 11 1.0
465000 ('D12_rnd_138_82',) 11 11 1.0
466000 ('D12_rnd_86_0',) 11 11 1.0
467000 ('D12_rnd_71_16',) 11 11 1.0
468000 ('D12_tex_7_0',) 11 26 0.82
469000 ('D12_tex_25_149',) 11 11 0.97
470000 ('D12_tex_30_85',) 11 17 0.88
Yes_Class 15
471000 ('D16_rnd_46_112',) 15 15 1.0
472000 ('D16_nat_39_158',) 15 15 1.0
473000 ('D16_tex_7_51'

In [16]:
csv_dir = img_dir = "Results_Merged/"
data_path = os.path.join(img_dir,'*csv')
files = glob.glob(data_path)

with open(os.path.join(csv_dir, 'Image_Level_Results.csv'), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Image Class Label","Number of Images",\
                         "Correct Predicted Images",\
                         "Number of Patches",\
                         "Total Patches(Correct Classified Images)",\
                         "Correct Predicted Patches(Correct Classified Images)",\
                         "Precetange Votes Per Image(Only Correct Images)",\
                         "Average Softmax Probability of Correct Patch(Only Correct Images)"])

In [17]:
for f in files:
      df = pd.read_csv(f)
      data = df.sort_values(by=['Patch_Filename'])
      classname = Path(f).stem
      classname = int(classname.split("_")[2])
      a="a"

      true_image_class = classname
      total_images_perclass = 0
      correct_images_perclass = 0
      total_patches_perclass = 0

      total_class_votes =0
      total_class_patches =0
      prob_avg_correct_patch = 0.0
      votes = 0

      arr_pred_patches = []
      arr_pred_patches_prob = []
      total_correc_img_patches = 0


      for ind in data.index:
              filename = df['Patch_Filename'][ind]
              pred_patch_class = df['Predicted Class'][ind]
              pred_patch_prob = df['Probability of Predicted Class'][ind]
              
              filename = filename.split("_")
              file_length = len(filename)
              initial_filename = filename[:-1]
              patch_name = filename[file_length-1]
              #print(initial_filename)
              #print(patch_name)

              if(a!= initial_filename and a=="a"):
                  a = initial_filename



              if(a!=initial_filename and a!="a"):
                  total_images_perclass = total_images_perclass + 1

                  counts = np.bincount(arr_pred_patches)
                  pred_image_class = np.argmax(counts)
                  votes = np.count_nonzero(arr_pred_patches==pred_image_class)
                  s=0
                  
                  if(pred_image_class == true_image_class):
                      correct_images_perclass = correct_images_perclass + 1
                      total_class_votes = total_class_votes + votes
                      total_class_patches = total_class_patches + len(arr_pred_patches)
                      
                      total_correc_img_patches = total_correc_img_patches + len(arr_pred_patches)

                      z = np.where(np.array(arr_pred_patches)==true_image_class,1,0)
                      for i in range(0,len(arr_pred_patches)):
                          if(z[i]==1):
                              s= s+1
                              prob_avg_correct_patch = prob_avg_correct_patch + arr_pred_patches_prob[i]
              
                  arr_pred_patches = []
                  arr_pred_patches_prob = []
                  a = initial_filename



              if(a==initial_filename):
                  total_patches_perclass = total_patches_perclass + 1
                  arr_pred_patches.append(pred_patch_class)
                  arr_pred_patches_prob.append(pred_patch_prob)



      total_images_perclass = total_images_perclass + 1

      counts = np.bincount(arr_pred_patches)
      pred_image_class = np.argmax(counts)
      votes = np.count_nonzero(arr_pred_patches==pred_image_class)

      if(pred_image_class == true_image_class):
          correct_images_perclass = correct_images_perclass + 1
          total_class_votes = total_class_votes + votes
          total_class_patches = total_class_patches + len(arr_pred_patches)
          
          total_correc_img_patches = total_correc_img_patches + len(arr_pred_patches)

          z = np.where(np.array(arr_pred_patches)==true_image_class,1,0)
          for i in range(0,len(arr_pred_patches)):
              if(z[i]==1):
                  prob_avg_correct_patch = prob_avg_correct_patch + arr_pred_patches_prob[i]

      arr_pred_patches = []
      arr_pred_patches_prob = []
      
      if(correct_images_perclass!=0):
          prob_avg_correct_patch = prob_avg_correct_patch / total_class_votes
          prob_avg_correct_patch = np.around(prob_avg_correct_patch,decimals=2)
          avg_vote_perclass = np.around((total_class_votes*100) / total_correc_img_patches,decimals=2)
          print(total_images_perclass,total_patches_perclass,correct_images_perclass,total_class_votes,avg_vote_perclass,prob_avg_correct_patch)  

          with open(os.path.join(csv_dir + 'Image_Level_Results.csv'), 'a+', newline='') as file:
              writer = csv.writer(file)
              writer.writerow([classname,total_images_perclass,correct_images_perclass,total_patches_perclass,\
                              total_correc_img_patches,total_class_votes,\
                              avg_vote_perclass,\
                              prob_avg_correct_patch])
      else:
          with open(os.path.join(csv_dir + 'Image_Level_Results.csv'), 'a+', newline='') as file:
              writer = csv.writer(file)
              writer.writerow([classname,total_images_perclass,correct_images_perclass,total_patches_perclass,\
                              total_correc_img_patches,total_class_votes,\
                              avg_vote_perclass,\
                              prob_avg_correct_patch])
              
              
df = pd.read_csv(os.path.join(csv_dir + 'Image_Level_Results.csv'))

ILA_merged = sum(df['Correct Predicted Images']) / sum(df['Number of Images']) * 100

print(f"Image Level Accuracy: {ILA_merged}%")

68 17408 62 14217 89.57 0.98
64 16384 60 14043 91.43 0.99
75 19200 71 17250 94.91 0.99
70 17920 46 11217 95.25 0.99
64 16384 63 15194 94.21 0.99
66 16896 58 13768 92.73 0.99
64 16384 64 16091 98.21 1.0
72 18432 70 16728 93.35 0.99
76 19456 70 16682 93.09 0.99
79 20224 78 18639 93.34 0.99
67 17152 48 11348 92.35 0.99
63 16128 62 14567 91.78 0.99
66 16896 63 14091 87.37 0.98
66 16896 65 15675 94.2 0.99
65 16640 63 15467 95.9 0.99
63 16128 53 11408 84.08 0.98
66 16896 56 13371 93.27 0.99
63 16128 54 10769 77.9 0.97
63 16128 55 13563 96.33 1.0
88 22528 84 20431 95.01 0.99
83 21248 80 20222 98.74 1.0
65 16640 59 13731 90.91 0.99
63 16128 47 10350 86.02 0.97
65 16640 63 14738 91.38 0.99
63 16128 48 9820 79.92 0.97
62 15872 55 10637 75.55 0.97
64 16384 60 14503 94.42 0.99
64 16384 52 11345 85.22 0.98
67 17152 65 15092 90.7 0.98
67 17152 55 13258 94.16 0.99
Image Level Accuracy: 90.05416051206302%


In [18]:
print(f"Image Level Accuracy (Trained on Random Set, Tested on Random Set): {ILA_random}%")
print(f"Image Level Accuracy (Trained on Random Set, Tested on Similar Set): {ILA_similar}%")
print(f"Image Level Accuracy (Trained on Random Set, Tested on Merged Set): {ILA_merged}%")

Image Level Accuracy (Trained on Random Set, Tested on Random Set): 98.8795518207283%
Image Level Accuracy (Trained on Random Set, Tested on Similar Set): 80.20833333333334%
Image Level Accuracy (Trained on Random Set, Tested on Merged Set): 90.05416051206302%
